In [1]:
import numpy as np
import time

In [4]:
flux, tot_var, models, rvecs, drvecs, av_init, rv_init, mcoeffs,\
  mtol, init_thresh, mags, mags_var, avlim, av_gauss,\
   rvlim, rv_gauss = np.load('optimize_fit.npy', allow_pickle=True)

In [5]:
def preprocess_get_seds(data, tot_var, models, rvecs, drvecs, av, rv, mag_coeffs,
                  avlim=(0., 20.), av_gauss=(0., 1e6),
                  rvlim=(1., 8.), rv_gauss=(3.32, 0.18),
                  resid=None, tol=0.05, init_thresh=5e-3, stepsize=1.,
                  mags=None, mags_var=None):
    """
    Optimize the distance and reddening between the models and the data using
    the gradient.

    Parameters
    ----------
    data : `~numpy.ndarray` of shape `(Nfilt)`
        Observed data values.

    tot_var : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        Associated (Normal) errors on the observed values compared to the
        models.

    models : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        Model predictions.

    rvecs : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        Associated model reddening vectors.

    drvecs : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        Associated differential model reddening vectors.

    av : `~numpy.ndarray` of shape `(Nmodel,)`
        Av values of the models.

    rv : `~numpy.ndarray` of shape `(Nmodel,)`
        Rv values of the models.

    mag_coeffs : `~numpy.ndarray` of shape `(Nmodel, Nfilt, 3)`
        Magnitude coefficients used to compute reddened photometry for a given
        model.

    avlim : 2-tuple, optional
        The lower and upper bound where the reddened photometry is reliable.
        Default is `(0., 20.)`.

    av_gauss : 2-tuple, optional
        The mean and standard deviation of the Gaussian prior that is placed
        on A(V). The default is `(0., 1e6)`, which is designed to be
        essentially flat over `avlim`.

    rvlim : 2-tuple, optional
        The lower and upper bound where the reddening vector shape changes
        are reliable. Default is `(1., 8.)`.

    rv_gauss : 2-tuple, optional
        The mean and standard deviation of the Gaussian prior that is placed
        on R(V). The default is `(3.32, 0.18)` based on the results from
        Schlafly et al. (2016).

    resid : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        Residuals between the data and models.
        If not provided, this will be computed.

    tol : float, optional
        The maximum tolerance in the computed Av and Rv values used to
        determine convergence during the magnitude fits. Default is `0.05`.

    init_thresh : bool, optional
        The weight threshold used to mask out fits after the initial
        magnitude-based fit before transforming the results back to
        flux density (and iterating until convergence). Default is `5e-3`.

    stepsize : float or `~numpy.ndarray`, optional
        The stepsize (in units of the computed gradient). Default is `1.`.

    mags : `~numpy.ndarray` of shape `(Nfilt)`, optional
        Observed data values in magnitudes.

    mags_var : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`, optional
        Associated (Normal) errors on the observed values compared to the
        models in magnitudes.

    Returns
    -------
    models_new : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        New model predictions. Always returned in flux densities.

    rvecs_new : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        New reddening vectors. Always returned in flux densities.

    drvecs_new : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        New differential reddening vectors. Always returned in flux densities.

    scale : `~numpy.ndarray` of shape `(Nmodel)`, optional
        The best-fit scale factor.

    Av : `~numpy.ndarray` of shape `(Nmodel)`, optional
        The best-fit reddening.

    Rv : `~numpy.ndarray` of shape `(Nmodel)`, optional
        The best-fit reddening shapes.

    icov_sar : `~numpy.ndarray` of shape `(Nmodel, 3, 3)`, optional
        The precision (inverse covariance) matrices expanded around
        `(s_ML, Av_ML, Rv_ML)`.

    resid : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        Residuals between the data and models.

    """

    # Compute residuals.
    if resid is None:
        if mags is not None and mags_var is not None:
            resid = mags - models
        else:
            resid = data - models

    Av_mean, Av_std = av_gauss
    Rv_mean, Rv_std = rv_gauss

    if mags is not None and mags_var is not None:
        # If magnitudes are provided, we can solve the linear system
        # explicitly for `(s_ML, Av_ML, r_ML=Av_ML*Rv_ML)`. We opt to
        # solve for Av and Rv in turn to so we can impose priors and bounds
        # on both quantities.

        # Compute constants.
        s_den = np.sum(1. / mags_var, axis=1)
        rp_den = np.sum(np.square(drvecs) / mags_var, axis=1)
        srp_mix = np.sum(drvecs / mags_var, axis=1)

        err = 1e300
        while err > tol:
            # Solve for Av.
            # Derive partial derivatives.
            a_den = np.sum(np.square(rvecs) / mags_var, axis=1)
            sa_mix = np.sum(rvecs / mags_var, axis=1)
            # Compute residual terms.
            resid_s = np.sum(resid / mags_var, axis=1)
            resid_a = np.sum(resid * rvecs / mags_var, axis=1)
            # Add in Gaussian Av prior.
            resid_a += (Av_mean - av) / Av_std**2
            a_den += 1. / Av_std**2
            # Compute determinants (normalization terms).
            sa_idet = 1. / (s_den * a_den - sa_mix**2)
            # Compute ML solution for Delta_Av.
            dav = sa_idet * (s_den * resid_a - sa_mix * resid_s)

            # Prevent Av from sliding off the provided bounds.
            dav_low, dav_high = avlim[0] - av, avlim[1] - av
            lsel, hsel = dav < dav_low, dav > dav_high
            dav[lsel] = dav_low[lsel]
            dav[hsel] = dav_high[hsel]

            # Increment to new Av.
            av += dav
            # Update residuals.
            resid -= dav[:, None] * rvecs  # update residuals

            # Solve for Rv.
            # Derive partial derivatives.
            r_den = rp_den * av**2
            sr_mix = srp_mix * av
            # Compute residual terms.
            resid_s = np.sum(resid / mags_var, axis=1)
            resid_r = np.sum(resid * drvecs / mags_var, axis=1) * av
            # Add in Gaussian Rv prior.
            resid_r += (Rv_mean - rv) / Rv_std**2
            r_den += 1. / Rv_std**2
            # Compute determinants (normalization terms).
            sr_idet = 1. / (s_den * r_den - sr_mix**2)
            # Compute ML solution for Delta_Rv.
            drv = sr_idet * (s_den * resid_r - sr_mix * resid_s)

            # Prevent Rv from sliding off the provided bounds.
            drv_low, drv_high = rvlim[0] - rv, rvlim[1] - rv
            lsel, hsel = drv < drv_low, drv > drv_high
            drv[lsel] = drv_low[lsel]
            drv[hsel] = drv_high[hsel]

            # Increment to new Rv.
            rv += drv
            # Update residuals.
            resid -= (av * drv)[:, None] * drvecs
            # Update reddening vector.
            rvecs += drv[:, None] * drvecs

            # Compute error based on best-fitting objects.
            chi2 = np.sum(np.square(resid) / mags_var, axis=1)
            logwt = -0.5 * chi2
            init_sel = np.where(logwt > np.max(logwt) + np.log(init_thresh))[0]
            err = np.max([np.abs(dav[init_sel]), np.abs(drv[init_sel])])
    else:
        # If our data is in flux densities, we can solve the linear system
        # implicitly for `(s_ML, Av_ML, Rv_ML)`. However, the solution
        # is not necessarily as numerically stable as one might hope
        # due to the nature of our Taylor expansion in flux.
        # Instead, it is easier to iterate in `(dAv, dRv)` from
        # a good guess for `(s_ML, Av_ML, Rv_ML)`. We opt to solve both
        # independently at fixed `(Av, Rv)` to avoid recomputing models.

        # Derive ML Delta_Av (`dav`) between data and models.
        a_num = np.sum(rvecs * resid / tot_var, axis=1)
        a_den = np.sum(np.square(rvecs) / tot_var, axis=1)
        a_num += (Av_mean - av) / Av_std**2  # add Av gaussian prior
        a_den += 1. / Av_std**2  # add Av gaussian prior
        dav = a_num / a_den
        # Adjust dAv based on the provided stepsize.
        dav *= stepsize

        # Derive ML Delta_Rv (`drv`) between data and models.
        r_num = np.sum(drvecs * resid / tot_var, axis=1)
        r_den = np.sum(np.square(drvecs) / tot_var, axis=1)
        r_num += (Rv_mean - rv) / Rv_std**2  # add Rv gaussian prior
        r_den += 1. / Rv_std**2  # add Rv gaussian prior
        drv = r_num / r_den
        # Adjust dRv based on the provided stepsize.
        drv *= stepsize

        # Prevent Av from sliding off the provided bounds.
        dav_low, dav_high = avlim[0] - av, avlim[1] - av
        lsel, hsel = dav < dav_low, dav > dav_high
        dav[lsel] = dav_low[lsel]
        dav[hsel] = dav_high[hsel]
        # Increment to new Av.
        av += dav

        # Prevent Rv from sliding off the provided bounds.
        drv_low, drv_high = rvlim[0] - rv, rvlim[1] - rv
        lsel, hsel = drv < drv_low, drv > drv_high
        drv[lsel] = drv_low[lsel]
        drv[hsel] = drv_high[hsel]
        # Increment to new Rv.
        rv += drv

    # Recompute models with new Rv.
    return mag_coeffs, av, rv
#     models, rvecs, drvecs = brutus_get_seds(mag_coeffs, av=av, rv=rv,
#                                      return_flux=True,
#                                      return_rvec=True, return_drvec=True)

In [28]:
mag_coeffs, av, rv = preprocess_get_seds(flux, tot_var, models, rvecs, drvecs, av_init, rv_init, mcoeffs,
              tol=mtol, init_thresh=init_thresh,
              resid=None, mags=mags, mags_var=mags_var,
              avlim=avlim, av_gauss=av_gauss,
              rvlim=rvlim, rv_gauss=rv_gauss);

# Get SEDs testing

In [18]:
from numba import jit

In [38]:
def brutus_get_seds(mag_coeffs, av=None, rv=None, return_flux=False,
             return_rvec=False, return_drvec=False):

    t0 = time.time()
    Nmodels, Nbands, Ncoef = mag_coeffs.shape
    if av is None:
        av = np.zeros(Nmodels)
    # elif isinstance(av, (int, float)):
    else:
        av = np.full(Nmodels, av)
    if rv is None:
        rv = np.full(Nmodels, 3.3)
    # elif isinstance(rv, (int, float)):
    else:
        rv = np.full(Nmodels, rv)
    t1 = time.time()
#     print('checkpoint 1:', t1-t0)
        
    t0 = time.time()
    # Turn provided Av values into polynomial features.
    mags = mag_coeffs[:, :, 0]
    r0 = mag_coeffs[:, :, 1]
    dr = mag_coeffs[:, :, 2]
    t1 = time.time()
#     print('checkpoint 2:', t1-t0)

    # Compute SEDs.
    t0 = time.time()
    drvecs = np.array(dr)
    rvecs = r0 + rv[:, None] * drvecs
    seds = mags + av[:, None] * rvecs
    t1 = time.time()
#     print('checkpoint 3:', t1-t0)

    # Convert to flux.
    t0 = time.time()
    if return_flux:
        seds = 10**(-0.4 * seds)
        if return_rvec:
            rvecs *= -0.4 * np.log(10.) * seds
        if return_drvec:
            drvecs *= -0.4 * np.log(10.) * seds
    t1 = time.time()
#     print('checkpoint 4:', t1-t0)

    if return_rvec and return_drvec:
        return seds, rvecs, drvecs
    elif return_rvec:
        return seds, rvecs
    elif return_drvec:
        return seds, drvecs
    else:
        return seds

In [9]:
import cProfile

In [30]:
cProfile.run("brutus_get_seds(mag_coeffs, av=av, rv=rv,\
                                     return_flux=True,\
                                     return_rvec=True, return_drvec=True)", "profile_get_seds.dat")

In [31]:
import pstats
from pstats import SortKey

In [32]:
p = pstats.Stats('profile_get_seds.dat')
# p.strip_dirs().sort_stats(-1).print_stats()

In [33]:
p.sort_stats(SortKey.TIME).print_stats()

Sat May 16 18:25:13 2020    profile_get_seds.dat

         17 function calls in 0.136 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.120    0.120    0.132    0.132 <ipython-input-16-6cf079e37e2b>:1(brutus_get_seds)
        3    0.009    0.003    0.009    0.003 {built-in method numpy.array}
        1    0.004    0.004    0.136    0.136 <string>:1(<module>)
        2    0.002    0.001    0.002    0.001 {built-in method numpy.core._multiarray_umath.implement_array_function}
        2    0.001    0.000    0.006    0.003 /usr/local/lib/python3.7/site-packages/numpy/core/numeric.py:283(full)
        1    0.000    0.000    0.136    0.136 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 {built-in method numpy.empty}
        2    0.000    0.000    0.002    0.001 <__array_function__ internals>:2(copyto)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' o

In [102]:
def get_seds(mag_coeffs, av=None, rv=None, return_flux=False,
             return_rvec=False, return_drvec=False):

    t0 = time.time()
#     mag_coeffs = np.array(mag_coeffs, order='F')
    t1 = time.time()
#     print('checkpoint 0:', t1-t0)
    
    t0 = time.time()
    Nmodels, Nbands, Ncoef = mag_coeffs.shape
    if av is None:
        av = np.zeros(Nmodels)
    # elif isinstance(av, (int, float)):
    else:
        av = np.full(Nmodels, av)
    if rv is None:
        rv = np.full(Nmodels, 3.3)
    # elif isinstance(rv, (int, float)):
    else:
        rv = np.full(Nmodels, rv)
    t1 = time.time()
#     print('checkpoint 1:', t1-t0)
        
    t0 = time.time()
    # Turn provided Av values into polynomial features.
    mags = mag_coeffs[:, :, 0]
    r0 = mag_coeffs[:, :, 1]
    dr = mag_coeffs[:, :, 2]
    t1 = time.time()
#     print('checkpoint 2:', t1-t0)

    # Compute SEDs.
    t0 = time.time()
    drvecs = np.asarray(dr)
    rvecs = r0 + rv[:, None] * drvecs
    seds = mags + av[:, None] * rvecs
    t1 = time.time()
#     print('checkpoint 3:', t1-t0)

    # Convert to flux.
    t0 = time.time()
    if return_flux:
#         seds = 10**(-0.4 * seds)
        seds = np.power(10., -0.4 * seds)
        if return_rvec:
            rvecs *= -0.4 * np.log(10.) * seds
        if return_drvec:
            drvecs *= -0.4 * np.log(10.) * seds
    t1 = time.time()
#     print('checkpoint 4:', t1-t0)

    if return_rvec and return_drvec:
        return seds, rvecs, drvecs
    elif return_rvec:
        return seds, rvecs
    elif return_drvec:
        return seds, drvecs
    else:
        return seds

In [96]:
seds, rvecs, dvecs = get_seds(mag_coeffs, av=av, rv=rv, return_flux=True, return_rvec=True, return_drvec=True)

checkpoint 0: 0.06918191909790039


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: overflow encountered in multiply


In [81]:
b_seds, b_rvecs, b_dvecs = brutus_get_seds(mag_coeffs, av=av, rv=rv, return_flux=True, return_rvec=True, return_drvec=True)

In [69]:
print(seds[0], b_seds[0])
print(rvecs[0], b_rvecs[0])
print(dvecs[0], b_dvecs[0])

[5.56224275e-08 1.51321935e-07 3.48865669e-07 5.30686299e-07
 5.73094703e-07] [5.56224275e-08 1.51321935e-07 3.48865669e-07 5.30686299e-07
 5.73094703e-07]
[-6.41885441e-08 -9.97080403e-08 -1.53466199e-07 -1.35366016e-07
 -9.03737318e-08] [-6.41885441e-08 -9.97080403e-08 -1.53466199e-07 -1.35366016e-07
 -9.03737318e-08]
[-0. -0. -0. -0. -0.] [0. 0. 0. 0. 0.]


In [40]:
%%timeit
brutus_get_seds(mag_coeffs, av=av, rv=rv, return_flux=True, return_rvec=True, return_drvec=True);

136 ms ± 6.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [103]:
%%timeit
get_seds(mag_coeffs, av=av, rv=rv, return_flux=True, return_rvec=True, return_drvec=True);

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in multiply


120 ms ± 2.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
cProfile.run("get_seds(mag_coeffs, av=av, rv=rv,\
                                     return_flux=True,\
                                     return_rvec=True, return_drvec=True)", "profile_get_seds_opt.dat")